# Creating structures in pyiron

This section gives a brief introduction about some of the tools available in pyiron to construct atomic structures. For the sake of compatibility, our structure class is written to be compatible with the popular Atomistic Simulation Environment package ([ASE](https://wiki.fysik.dtu.dk/ase/)). This makes it possible to use routines from ase to help set-up structures. Furthermore pyiron uses the [NGLview](http://nglviewer.org/nglview/latest/api.html) package to visualize the structures and trajectories

As preparation for the following discussion we import a few python modules and create a project. 

In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pylab as plt
from pyiron.project import Project

pr = Project('structures')

## Bulk crystals

In this section we discuss various possibilities to create bulk crystal structures.

### Using `create_structure()`

The simplest way to generate simple crystal structures is using the inbuilt `create_structure()` function specifying the element symbol, Bravais basis and the lattice constant(s)

Note: The output gives a cubic cell rather than the smallest non-orthogonal unit cell.

In [2]:
basis = pr.create_structure('Al', 
                            bravais_basis='fcc', 
                            lattice_constant=4.05)
basis.plot3d()

NGLWidget()

### Using `create_ase_bulk()`

Another convenient way to set up structures is using the `create_ase_bulk()` function which is built on top of the ASE build package for [bulk crystals](https://wiki.fysik.dtu.dk/ase/ase/build/build.html#ase.build.bulk). This function returns an object which is of the pyiron structure object type

**Example:** fcc bulk aluminum in a cubic cell

In [3]:
basis = pr.create_ase_bulk('Al', cubic=True)
basis.plot3d()

NGLWidget()

**Example:** wurtzite GaN in a 3x3x3 repeated orthorhombic cell.

Note: In contrast to new_basis = basis.repeat() which creates a new object, set_repeat() modifies the existing structure object.

In [4]:
basis = pr.create_ase_bulk('AlN', 
                           crystalstructure='wurtzite', 
                           a=3.5, orthorhombic=True)
basis.set_repeat([3,3,3])
basis.plot3d(spacefill=False)

NGLWidget()

### Using the ASE spacegroup class

In [5]:
from ase.spacegroup import crystal
from pyiron_atomistics.structure.atoms import ase_to_pyiron

a = 9.04
skutterudite = crystal(('Co', 'Sb'),
                       basis=[(0.25, 0.25, 0.25), (0.0, 0.335, 0.158)],
                       spacegroup=204,
                       cellpar=[a, a, a, 90, 90, 90])
skutterudite = ase_to_pyiron(skutterudite)

In [6]:
skutterudite.plot3d()

NGLWidget()

## Point defects

### Vacancy

In [7]:
basis = pr.create_ase_bulk('Al', cubic=True)
basis.set_repeat([4,4,4])
del basis[0]
basis.plot3d()

NGLWidget()

Random set of n$_{vac}$ vacancies

In [8]:
n_vac = 24
basis = pr.create_ase_bulk('Al', cubic=True)
basis.set_repeat([4,4,4])
del basis[np.random.permutation(len(basis))[:n_vac]]
print (len(basis))
basis.plot3d()

232


NGLWidget()

### Random substitutial alloys

In [9]:
n_sub = 24
basis = pr.create_ase_bulk('Al', cubic=True)
basis.set_repeat([4,4,4])
basis[np.random.permutation(len(basis))[:n_sub]] = 'Mg'
print (len(basis))
basis.plot3d()

256


NGLWidget()

## Explicit definition of the structure

You can also set-up structures through the explicit input of the cell parameters and positions

In [10]:
cell = 10.0 * np.eye(3) # Specifying the cell dimensions
positions = [[0.25, 0.25, 0.25], [0.75, 0.75, 0.75]]
elements = ['O', 'O']

# Now use the Atoms class to create the instance.
O_dimer = pr.create_atoms(elements=elements, scaled_positions=positions, cell=cell)

O_dimer.plot3d()

NGLWidget()

## Surfaces (with ASE)

Popular surfaces can also be defined with the `create_surface()` function based on the [ASE surface builder](https://wiki.fysik.dtu.dk/ase/ase/build/surface.html)

In [11]:
fcc111 = pr.create_surface("Pt", 
                           surface_type="fcc111", 
                           size=(3, 4, 5), 
                           a=4.2, 
                           vacuum=20)

In [12]:
fcc111.plot3d()

NGLWidget()

## Importing from cif/other file formats

Parsers from ASE can be used to import structures from other formats. In this example, we will download and import a Nepheline structure from the [Crystallography Open Database (COD)](http://www.crystallography.net/cod/index.php)

In [13]:
import urllib
filename = '1008753.cif'
url = 'http://www.crystallography.net/cod/{}'.format(filename)

In [14]:
urllib.request.urlretrieve(url=url, filename='strucs.'+filename);

In [15]:
# Using ase parsers to read the structure and then convert to a pyiron instance
import ase
from pyiron_atomistics.structure.atoms import ase_to_pyiron

structure = ase_to_pyiron(ase.io.read(filename='strucs.'+filename,
                                      format='cif'))

In [16]:
structure.plot3d()

NGLWidget()